In [3]:
# Enter your Email ID, Password and Designation in the input field after executing the script

# import libraries
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings('ignore')


# webdriver for Chrome - chromedriver
driver = webdriver.Chrome()

driver.maximize_window()

# sticky timeout to implicitly wait for an element to be found
driver.implicitly_wait(10)

# open url in the browser
driver.get('https://www.naukri.com/')

driver.find_element('xpath', '//*[@id="login_Layer"]').click()
    
eid = driver.find_element("xpath",'//*[@id="root"]/div[4]/div[2]/div/div/div[2]/div/form/div[2]/input')
email_id = input('ENTER YOUR EMAIL-ID: ')   
eid.send_keys(email_id)
    
    
epass = driver.find_element("xpath",'//*[@id="root"]/div[4]/div[2]/div/div/div[2]/div/form/div[3]/input')
password = input('ENTER YOUR PASSWORD: ')  
epass.send_keys(password)
    
driver.find_element('xpath', '//*[@id="root"]/div[4]/div[2]/div/div/div[2]/div/form/div[6]/button').click()
    
    

# enter job role - data scientist
search = driver.find_element("xpath",'/html/body/div[3]/div[2]/div[1]/div/span')
search.click()


role = driver.find_element("xpath",'/html/body/div[3]/div[2]/div[1]/div/div/div[1]/div/div/div/input')
designation = input('ENTER DESIGNATION YOU ARE APPLYING FOR: ')
role.send_keys(designation)

# location - India
location = driver.find_element("xpath",
                               '/html/body/div[3]/div[2]/div[1]/div/div/div[5]/div/div/div/input')
location.send_keys('india')

# click search
driver.find_element('xpath', '/html/body/div[3]/div[2]/div[1]/div/button').click()


# new dataframe fwith required columns
df = pd.DataFrame(columns=['Job_Role', 'Company',
                  'Location', 'Job Experience', 'Skills/Description'])
df

# helper function

# Function to convert  a list to string
def listToString(s):

    # initialize an empty string
    str1 = ", "

    # return string
    return (str1.join(s))


# page number
page = 1

condition = True

while condition:

    # atmost 20 results per page
    
    for i in range(1, 21):

        try:

            role = driver.find_element("xpath",
                                       '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/div/a'.format(i)).text
            company = driver.find_element("xpath",
                                          '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/div/div/a[1]'.format(i)).text
            salary = driver.find_element("xpath",
                                         '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/ul/li[2]/span[1]'.format(i)).text
            location = driver.find_element("xpath",
                                           '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/ul/li[3]/span'.format(i)).text
            experience = driver.find_element("xpath",'//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/ul/li[1]/span[1]'.format(i)).text.replace(' Yrs', '')
            
            skill_lst = driver.find_element("xpath",
                                            '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/ul'.format(i)).text.split('\n')

            skill = listToString(skill_lst)

            # append collected information to the dataframe df
            
            exp = experience.split('-')
            
            if exp[0].isdigit():
                
                if int(exp[0]) < 2 and int(exp[1]) <= 10:

                    wait = WebDriverWait(driver, 10)

                    # Store the ID of the original window
                    original_window = driver.current_window_handle

                    # Check we don't have other windows open already
                    assert len(driver.window_handles) == 1 
                    
                    # Click the link which opens in a new window
                    # click to apply
                    driver.find_element("xpath",
                                           '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[2]/article[{}]/div[1]/div/a'.format(i)).click()

                    
                    
                    # Wait for the new window or tab
                    wait.until(EC.number_of_windows_to_be(2))

                    # Loop through until we find a new window handle
                    for window_handle in driver.window_handles:
                        if window_handle != original_window:
                            driver.switch_to.window(window_handle)

                            try:
                                # sticky timeout to implicitly wait for an element to be found
                                driver.implicitly_wait(10)
                                
                                driver.find_element("xpath",'//*[@id="apply-button"]'.format(i)).click()
                                time.sleep(20)
                                driver.close()                       
                                #Switch back to the old tab or window
                                driver.switch_to.window(original_window)
                                break

                            except:
                                driver.close()                       
                                #Switch back to the old tab or window
                                driver.switch_to.window(original_window)


                    df.loc[len(df.index)] = [role, company,location, experience, skill]
                    
            else:                
                continue
                
                
        except NoSuchElementException:
            pass

    page += 1

    # click next
    nxt = driver.find_element("xpath",
                              '//*[@id="root"]/div[5]/div[2]/div/section[2]/div[3]/div/a[2]')
    print(nxt)
    driver.execute_script("arguments[0].click();", nxt)

    # total 12700+ results showed, 20 results per page => 600+ pages

    # stop after 620 pages
    if page >= 15:
        condition = False

# drop duplicates
df = df.drop_duplicates()

# shape of the dataframe
df.shape

# selecting first 12,000 results
df = df.iloc[:12000, :]
df.shape

# first five rows
df.head()

# last five rows
df.tail()

# # save as csv file
df.to_excel('naukri_'+designation+'_jobs_india.xlsx', index=False)

print(df)

Enter your Email ID: shubham.wankar.official@gmail.com
Enter your password: python_developer
Enter  Designationpython developer
<selenium.webdriver.remote.webelement.WebElement (session="ca47f6b52c084f2ea3cd0d9aedadce56", element="08A5E3C64674A4A46EAFB4E90A2A0B27_element_490")>


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.141)
Stacktrace:
	GetHandleVerifier [0x00007FF744C052A2+57122]
	(No symbol) [0x00007FF744B7EA92]
	(No symbol) [0x00007FF744A4E3AB]
	(No symbol) [0x00007FF744A2E1B9]
	(No symbol) [0x00007FF744AAB417]
	(No symbol) [0x00007FF744AB0D82]
	(No symbol) [0x00007FF744AA6FA0]
	(No symbol) [0x00007FF744A7D2B1]
	(No symbol) [0x00007FF744A7E494]
	GetHandleVerifier [0x00007FF744EAEF82+2849794]
	GetHandleVerifier [0x00007FF744F01D24+3189156]
	GetHandleVerifier [0x00007FF744EFACAF+3160367]
	GetHandleVerifier [0x00007FF744C96D06+653702]
	(No symbol) [0x00007FF744B8A208]
	(No symbol) [0x00007FF744B862C4]
	(No symbol) [0x00007FF744B863F6]
	(No symbol) [0x00007FF744B767A3]
	BaseThreadInitThunk [0x00007FFC1B7D257D+29]
	RtlUserThreadStart [0x00007FFC1D8CAA68+40]
